In [3]:
#!/usr/bin/env python3
# coding=utf-8
"""A simple example demonstrating how to use Argparse to support subcommands.
This example shows an easy way for a single command to have many subcommands, each of which takes different arguments
and provides separate contextual help.
"""
import argparse
import cmd2
import newspaper as np
from dateutil.parser import parse
from roundup_db1 import Entry, Category, Keyword, Publication, Author, Section, DataAccessLayer
import app,roundup_help
import sys, glob, datetime, os
import BTCInput2 as btc
import itertools as it
import warnings, functools, pprint

In [10]:
###SEARCH COMMAND

# create the top-level parser for the base command

# create the top-level parser for the search command
search_parser = argparse.ArgumentParser()
search_subparsers = search_parser.add_subparsers(title='subcommands', help='subcommand help')

# create the parser for the "entry" subcommand
parser_entry = search_subparsers.add_parser('entry', help='entry help')
parser_entry.add_argument('-id', '--entry_id', help='id of the entry')
parser_entry.add_argument('-idr', '--id_range', nargs=2,
                              help='minimum and maximum ids', )
parser_entry.add_argument('-d', '--date', help='date of entry')
parser_entry.add_argument('-l', '--url', help='article_url')
parser_entry.add_argument('-t', '--title', nargs='*', help='entry name')
parser_entry.add_argument('-dr', '--date_range', nargs=2, help='start date, end date')
parser_entry.add_argument('-c', '--category_id', help='category id')
parser_entry.add_argument('-desc',  '--description', nargs='*',
                           help='article description (optional)')

#create the parser for the "category" subcommand
parser_category = search_subparsers.add_parser('category', help='category help')
parser_category.add_argument('-id', '--category_id', help='category id')
parser_category.add_argument('-n', '--category_name', nargs='*', help='category name e.g. Python')
parser_category.add_argument('-s', '--section_id', help='section id')


#create the parser for the "section" subcommand
parser_section = search_subparsers.add_parser('section', help='section help')
parser_section.add_argument('-id', '--section_id', help='section id')

###ADD COMMAND
qa_parser = argparse.ArgumentParser()
qa_subparsers = qa_parser.add_subparsers(title='subcommands', help='subcommand help')

#create the parser for the "entry" subcommand

parser_add_entry = qa_subparsers.add_parser('entry', help='entry help')
parser_add_entry.add_argument('-l', '--url', help='url of article')
parser_add_entry.add_argument('-c', '--category_id', help='category id e.g. Comoros is 1')
parser_add_entry.add_argument('-d','--date', help='article date')
parser_add_entry.add_argument('-desc',  '--description', nargs='*',
                           help='article description (optional)')

##EXPORT COMMAND: parsers for the commands

exp_parser = argparse.ArgumentParser() #different from the exportparser
exp_subparsers = exp_parser.add_subparsers(title='subcommands', help='subcommand help')

#create the parser for the "docx" subcommand

docx_parser = exp_subparsers.add_parser('docx', help='docx help')
docx_parser.add_argument('-t', '--title', nargs='*', help='roundup title')
docx_parser.add_argument('-f',
        '--filename', help='filename (same directory as the app)')
docx_parser.add_argument('-r', '--date_range', nargs=2,
                                 help='search the dates between the start and end dates')

#create the parser for the "html" subcommand

html_parser = exp_subparsers.add_parser('html', help='docx help')
html_parser.add_argument('-t', '--title', nargs='*', help='roundup title')
html_parser.add_argument('-f',
        '--filename', help='filename (same directory as the app)')
html_parser.add_argument('-r', '--date_range', nargs=2,
                                 help='search the dates between the start and end dates')

class RoundupCMD(cmd2.Cmd):
    """
    Example cmd2 application where we a base command which has a couple subcommands
    and the "sport" subcommand has tab completion enabled.
    """
    def __init__(self):
        super().__init__()
        
    #add article section
    
    def base_ent(self, args):
        '''Adds an article with a single type of the command line. 
        quick_add url cat_id date'''
        if not args.description:
            app.qa(session=a.d.session, url=args.url, category_id=args.category_id, date=args.date)
        else:
            app.qa(session=a.d.session, url=args.url, category_id=args.category_id,
                   date=args.date, description=' '.join(args.description))
    
    parser_add_entry.set_defaults(func=base_ent)
    
    @cmd2.with_argparser(qa_parser)
    def do_qa(self, args):
        """Search command help"""
        func = getattr(args, 'func', None)
        if func is not None:
            # Call whatever subcommand function was selected
            func(self, args)
        else:
            # No subcommand was provided, so call help
            self.do_help('search')
    
    #search section
    
    def base_category(self, args):
        '''Generic category search function'''
        app.find_category(session=a.d.session, args=args)
        
    
    def base_entry(self, args):
        '''Generic entry search function'''
        app.find_entry(session=a.d.session, args=args)
    
    def base_section(self, args):
        '''Generic section search function'''
        pass
        
    parser_category.set_defaults(func=base_category)
    parser_entry.set_defaults(func=base_entry)

    @cmd2.with_argparser(search_parser)
    def do_search(self, args):
        """Search command help"""
        func = getattr(args, 'func', None)
        if func is not None:
            # Call whatever subcommand function was selected
            func(self, args)
        else:
            # No subcommand was provided, so call help
            self.do_help('search')
            
    #export section 
    #create export function with html and docx as subcommands
    
    def base_html(self, args):
        '''Generic export html function'''
        app.export_html2(session=a.d.session, program=args.filename,
                        start_date=parse(args.date_range[0]).date(),
                         end_date=parse(args.date_range[1]).date(),
                        title=' '.join(args.title))
        
    def base_docx(self, args):
        '''Generic export html function'''
        app.create_docx_roundup(args)
    
    html_parser.set_defaults(func=base_html)
    docx_parser.set_defaults(func=base_docx)
    
    @cmd2.with_argparser(exp_parser)
    def do_exp(self, args):
        """Search command help"""
        func = getattr(args, 'func', None)
        if func is not None:
            # Call whatever subcommand function was selected
            func(self, args)
        else:
            # No subcommand was provided, so call help
            self.do_help('exp')
            
    #create universal add entry functionality
            
    finalize_parser = argparse.ArgumentParser()
    finalize_parser.add_argument('-d', '--date', help='search a single date')
    finalize_parser.add_argument('-r', '--date_range', nargs=2,
                                 help='search the dates between the start and end dates')
    
    
    @cmd2.with_argparser(finalize_parser)
    def do_finalize(self, args):
        '''searches for articles without descriptions and lets user edit them'''
        if args.date:
            app.finalize2(session=a.d.session, start_date =args.date, end_date=args.date)
        elif args.date_range:
            app.finalize2(session=a.d.session, start_date = args.date_range[0],
                          end_date=args.date_range[1])
        else:
            print('Please enter date or date range. Check help for details')
            return
        
    def do_export_docx(self, line):
        '''Exports a roundup in docx form. The "line" argument is deleted by the function'''
        app.create_docx_roundup(line)
    
    export_parser = argparse.ArgumentParser()
    export_parser.add_argument('-t', '--title', nargs='*', help='roundup title')
    export_parser.add_argument('-f',
        '--filename', help='filename (same directory as the app)')
    export_parser.add_argument('-r', '--date_range', nargs=2,
                                 help='search the dates between the start and end dates')
    
    @cmd2.with_argparser(export_parser)
    def do_export_html(self, args):
        '''Export an html version of the roundup'''
        warnings.warn('export_html implementation in testing phase')
        app.export_html2(session=a.d.session, program=args.filename,
                        start_date=parse(args.date_range[0]).date(),
                         end_date=parse(args.date_range[1]).date(),
                        title=' '.join(args.title))
        
    display_parser = argparse.ArgumentParser()
    display_parser.add_argument('-s', '--section_id', help='section_id')
    
    @cmd2.with_argparser(display_parser)
    def do_display_categories(self, args):
        '''Displays category names for user convenience'''
        try:
            if args.section_id.isnumeric() == True:
                app.display_categories(section_id=args.section_id)
        except AttributeError:
            app.display_categories()
        
    def do_display_sections(self, line):
        '''Enter without a prefix, displays a list of all sections in the database'''
        app.display_sections()
    
    del_parser = argparse.ArgumentParser()
    del_parser.add_argument('-ty', '--item_type',
                            help='item type e.g. entry, category, section, author, publication')
    del_parser.add_argument('-id', '--id_value', help='item id')
    
    @cmd2.with_argparser(del_parser)
    def do_delete_item(self, args):
        '''Delete item '''
        warnings.warn('Unified delete function not fully tested')
        app.delete_item(session=a.d.session, model=args.item_type, id_value=args.id_value)
    
    
    def do_exit(self, arg):
        '''Exits the program, any existing database connections will be closed'''
        a.close()
        print('Exiting Roundup Generator')
        return True

In [ ]:
if __name__ == '__main__':
    #import sys
    #app = SubcommandsExample()
    #sys.exit(app.cmdloop())
    a=app.App()
    a.setup()
    RoundupCMD().cmdloop()

-f is not a recognized command, alias, or macro
/Users/thomassullivan/Library/Jupyter/runtime/kernel-dfea2eac-15b6-4aac-8446-a642b582350b.json is not a recognized command, alias, or macro


 help qa


usage: qa [-h] {entry} ...

Search command help

optional arguments:
  -h, --help  show this help message and exit

subcommands:
  {entry}     subcommand help
    entry     entry help


 help qa entry


usage: qa entry [-h] [-l URL] [-c CATEGORY_ID] [-d DATE]
                [-desc [DESCRIPTION [DESCRIPTION ...]]]

optional arguments:
  -h, --help            show this help message and exit
  -l URL, --url URL     url of article
  -c CATEGORY_ID, --category_id CATEGORY_ID
                        category id e.g. Comoros is 1
  -d DATE, --date DATE  article date
  -desc [DESCRIPTION [DESCRIPTION ...]], --description [DESCRIPTION [DESCRIPTION ...]]
                        article description (optional)


 display_categories


Categories: 
Category(id='1' name='skills', section=4)
Category(id='2' name='scripting/automation', section=1)
Category(id='3' name='Languages', section=4)
Category(id='4' name='Careers', section=1)


 qa entry -l https://www.theregister.co.uk/2020/03/02/redmonk_programming_languages/ -c 3 -d 03/02/2020 -desc Python, Java, and Javascript are three of the top languages, and the newer languages such as Go are well behind.



Title is being added...
("If you're writing code in Python, JavaScript, Java and PHP, relax. The hot "
 'trendy languages are still miles behind, this survey says')
None
Publication(publication_id='8' title='https://www.theregister.co.uk')
If you're writing code in Python, JavaScript, Java and PHP, relax. The hot trendy languages are still miles behind, this survey says added successfully


In [10]:
a.d.session.close()